In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

# WritingProject

## 长文内容结构

In [2]:
from langchain.pydantic_v1 import BaseModel, Field, root_validator
from typing import Any, Dict, Iterator, List, Optional, Union
import datetime
import random

global_id = 0

def create_content_id():
    global global_id
    now = datetime.datetime.now()
    random_digits = random.randint(100, 999)  # 生成一个四位的随机数

    # 更新 global_id
    global_id += 1
    if global_id > 99999:
        global_id = 0
    global_id_str = str(global_id).zfill(5)

    return now.strftime(f"%Y%m%d.%H%M%S.{global_id_str}.{random_digits}")
    
class BaseContent(BaseModel):
    id: Optional[str] = None
    topic: Optional[str] = None
    path: Optional[str] = None

    def load(self):
        pass  # 实现加载逻辑

    def save(self):
        pass  # 实现保存逻辑

    @root_validator
    def auto_generate_id(cls, value):
        if value['id'] is None:
            value['id'] = create_content_id()   
        return value

class DetailContent(BaseContent):
    text: Optional[str] = None

class OutlineContent(BaseContent):
    children: List[BaseContent] = []

    def add_content(self, content: BaseContent):
        if self.children is None:
            self.children = []
        self.children.append(content)

        return content.id

    def get_content(self, content_id: str) -> Optional[BaseContent]:
        """递归查询并返回指定id的Content"""
        if self.id == content_id:
            return self

        for child in self.children or []:
            if child.id == content_id:
                return child
            if isinstance(child, OutlineContent):
                found = child.get_content(content_id)
                if found:
                    return found

        return None

class Article(OutlineContent):
    task_content_id: Optional[str] = Field(None, description="The content ID of the task")

In [3]:
root = OutlineContent()

t1 = root.add_content(OutlineContent(topic="H1"))
t2 = root.add_content(OutlineContent(topic="H2"))
# t1.add_content(DetailContent(text="my headline 1"))
# t2.add_content(DetailContent(text="my headline 2"))

In [4]:
h1 = root.get_content(t1)
h2_id = h1.add_content(DetailContent(text="my headline 1"))
root.get_content(h2_id)

DetailContent(id='20240504.211537.00004.184', topic=None, path=None, text='my headline 1')

## 长文写作智能体

In [74]:
from langchain.pydantic_v1 import BaseModel, Field, root_validator
from langchain_zhipu import ChatZhipuAI
from langchain.prompts import ChatPromptTemplate
import json
import re

TASK_INSTRUCTIONS = """
你是一名优秀的写手，可以构思写作思路、扩展写作提纲、细化段落内容，

请务必记住：
1. 用户的提问一般都是写作任务，除非有明确的其他意图。
2. 当你收到新的写作任务，你应当做两种选择，要么输出写作提纲，要么输出细化的写作内容。
3. 你有输出300字的输出限制。
4. 你必须注意解读用户的提问中包含的「字数估计」、「字数xxx」等字眼，并作为用户要求的字数对待，如果你发现用户要求的字数超出限制，你就必须输出写作提纲；反之，你就输出段落内容。

例如：
用户要求写作的字数超出你的限定，一篇1500字左右，此时超出了300字的写作限定，你必须输出写作提纲，可以分为5个部份，每部份约300字左右；
用户要求写一篇80字左右，此时符合300字左右的限定，你必须输出为段落内容。

如果你决定输出「写作提纲」，就请按照如下格式输出写作大纲：
```json
{
    "类型": "outlines",
    "大纲列表": [
        {"标题名称": "标题名称", "字数估计": "字数估计", "内容摘要": "内容摘要"},
        {"标题名称": "标题名称", "字数估计": "字数估计", "内容摘要": "内容摘要"},
        （更多行）
        {"标题名称": "标题名称", "字数估计": "字数估计", "内容摘要": "内容摘要"}
    ],
    "大纲数量": "与以上列表相符的大纲数量"
}
```

如果你决定输出「段落内容」，就请按照如下格式输出：
```json
{
    "type": "paragraph",
    "text": "你的详细输出"
}
```

只输出上述的JSON内容即可，其他不必输出。
"""

def get_input(prompt: str = "\n👤: ") -> str:
    return input(prompt)

class WritingProject(BaseModel):
    """写作控制参数"""    
    words_per_step = 300
    words_all_limit = 1000

    # 任务类型可以包括：
    # - root     最初的写作规划
    # - outlines 写作大纲
    # - detail   细节内容
    task_type = ""

    def next_step(self):
        pass

    
    def ask_user(self, ai_said: str = "") -> tuple:
        resp = get_input()
        content = ""
        command = ""
    
        if(resp == "quit"):
            command = "quit"
        elif(resp == "ok"):
            content = ai_said
            command = "ok"
        else:
            content = resp
            command = "chat"
    
        return content, command

    def get_chain(self):
        prompt = ChatPromptTemplate.from_messages([
            ("system", TASK_INSTRUCTIONS),
            ("user", "{{input}}")
        ], template_format='jinja2')
        llm = ChatZhipuAI()
        return (prompt | llm)

    def run(self):
        # 初始化链
        chain = self.get_chain()
        ai_said = ""

        while True:
            # 用户输入
            resp, command = self.ask_user(ai_said)
            if command == "quit":
                print("-"*20, "quit")
                break
            elif command == "ok":
                # 生成目录或文件

                # 评估任务
                # 如果所有任务都OK，就结束
                
                # 任务分派
                # - 转移到新的子主题
                # - 重新分配session_id
                chain = self.get_chain()
                print("-"*20, "ok")
                continue
            else:
                # AI推理
                message = chain.invoke(resp)
                if len(message.content) > 0:
                    try:
                        json_match = re.search(r'```json\n(.*?)\n```', message.content, re.DOTALL)
                        if json_match:
                            json_string = json_match.group(1)
                            # 解析JSON字符串
                            data = json.loads(json_string)
                            print("\n🤖:", data)
                            continue
                    except json.JSONDecodeError as e:
                        print(f"JSON解析错误: {e}")
            print(message)


In [75]:
wp = WritingProject()
wp.run()


👤:  请帮我写一篇1000字半年度总结



🤖: {'类型': 'outlines', '大纲列表': [{'标题名称': '一、引言', '字数估计': '100字', '内容摘要': '简要回顾半年的工作生活，表明写总结的目的'}, {'标题名称': '二、工作总结', '字数估计': '400字', '内容摘要': '详细总结半年度的工作成果、经验及不足'}, {'标题名称': '三、个人成长', '字数估计': '300字', '内容摘要': '分析半年来自己在工作、学习、能力等方面的提升'}, {'标题名称': '四、问题与反思', '字数估计': '200字', '内容摘要': '反思半年工作中遇到的问题和困难，分析原因及改进措施'}, {'标题名称': '五、展望未来', '字数估计': '100字', '内容摘要': '对未来半年的工作目标和期望进行规划'}], '大纲数量': '5'}



👤:  帮我按要求编写：{'标题名称': '一、引言', '字数估计': '100字', '内容摘要': '简要回顾半年的工作生活，表明写总结的目的'}



🤖: {'类型': 'outlines', '大纲列表': [{'标题名称': '一、引言', '字数估计': '100字', '内容摘要': '简要回顾半年的工作生活，表明写总结的目的'}], '大纲数量': '1'}



👤:  请帮我按这个要求写：{'标题名称': '一、引言', '字数估计': '100字', '内容摘要': '简要回顾半年的工作生活，表明写总结的目的'}


...


🤖: {'type': 'paragraph', 'text': '二、工作总结\n在过去半年的工作中，我们团队取得了显著的成绩，但也存在一些不足。在项目实施方面，我们严格按照时间节点推进，确保了项目的顺利进行。在团队协作方面，大家相互支持，共同进步，提高了工作效率。然而，我们也发现，在某些环节中，沟通不畅和资源分配不均的问题影响了工作进度。在今后的工作中，我们将总结经验，改进方法，努力提高工作质量，以期更好地完成年度目标。'}



👤:  ok


-------------------- ok



👤:  quit


-------------------- quit
